In [1]:
# import everything and define a test runner function
from importlib import reload
from helper import run_test

import block
import ecc
import helper
import network
import script
import tx

### Exercise 8

#### 8.1. Verify a Merkle Proof
Transaction Hash:
```
e8270fb475763bc8d855cfe45ed98060988c1bdcad2ffc8364f783c98999a208
```

Index:
```
6
```

Merkle Root:
```
4297fb95a0168b959d1469410c7527da5d6243d99699e7d041b7f3916ba93301
```

Merkle Proof:
```
9ed0a5430b5b530822b1ce1b2b9a03d513c888aaa3f028f041bf143efd8c1b92
1dc4b438b3a842bcdd46b6ea5a4aac8d66be858b0ba412578027a1f1fe838c51
156f3662b07aaa4a0d9762faaa8c18afe4c211ff92eb1eae1952aa66627bbf2e
524c93c6dd0874c5fd9e4e57cfe83176e3c2841c973afb4043d225c22cc52983
```

#### 8.2. Make [this test](/edit/session7/block.py) pass.
```
block.py:BlockTest:test_verify_merkle_proof
```

In [23]:
# Exercise 8.1

from block import Proof
from helper import double_sha256, merkle_root, merkle_path, merkle_parent

tx_hash = bytes.fromhex('e8270fb475763bc8d855cfe45ed98060988c1bdcad2ffc8364f783c98999a208')
merkle_root = bytes.fromhex('4297fb95a0168b959d1469410c7527da5d6243d99699e7d041b7f3916ba93301')
proof_hex_hashes = [
    '9ed0a5430b5b530822b1ce1b2b9a03d513c888aaa3f028f041bf143efd8c1b92',
    '1dc4b438b3a842bcdd46b6ea5a4aac8d66be858b0ba412578027a1f1fe838c51',
    '156f3662b07aaa4a0d9762faaa8c18afe4c211ff92eb1eae1952aa66627bbf2e',
    '524c93c6dd0874c5fd9e4e57cfe83176e3c2841c973afb4043d225c22cc52983',
]
proof_hashes = [bytes.fromhex(x) for x in proof_hex_hashes]
index = 6

# get the current hash (reverse tx_hash as it needs to be little endian)
current = tx_hash[::-1]
# initialize the current_index to the index
current_index = index
# loop through proof hashes
for proof_hash in proof_hashes:
    # if current_index is odd, proof_hash is on the left
    if current_index % 2 == 1:
        current = merkle_parent(proof_hash, current)
    # if current_index is even, proof_hash is on the right
    else:
        current = merkle_parent(current, proof_hash)
    # update current_index to be integer divide by 2
    current_index //= 2
# see if the current one reversed is the same as merkle root
print(current[::-1] == merkle_root)

True


In [24]:
# Exercise 8.2

reload(block)
run_test(block.BlockTest('test_verify_merkle_proof'))

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


### Exercise 9

#### Check the cheat sheet for the network message structure.

#### 9.1. Parse this message
```
f9beb4d976657261636b000000000000000000005df6e0e2
```

#### 9.2. Make [these test](/edit/session7/network.py) pass.
```
network.py:NetworkTest:test_parse
network.py:NetworkTest:test_serialize
```

In [25]:
# Exercise 9.1

msg = bytes.fromhex('f9beb4d976657261636b000000000000000000005df6e0e2')

# first 4 are network magic
magic = msg[:4]
# next 12 are command
command = msg[4:16]
# next 4 are payload length
payload_length = msg[16:20]
# next 4 are checksum
checksum = msg[20:24]
# rest is payload
payload = msg[24:]
print(command)

b'verack\x00\x00\x00\x00\x00\x00'


In [26]:
# Exercise 9.2

reload(network)
run_test(network.NetworkEnvelopeTest('test_parse'))
run_test(network.NetworkEnvelopeTest('test_serialize'))

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK
.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


### Exercise 10

#### 10.1. Parse this message
```
f9beb4d974780000000000000000000002010000e293cdbe01000000016dbddb085b1d8af75184f0bc01fad58d1266e9b63b50881990e4b40d6aee3629000000008b483045022100f3581e1972ae8ac7c7367a7a253bc1135223adb9a468bb3a59233f45bc578380022059af01ca17d00e41837a1d58e97aa31bae584edec28d35bd96923690913bae9a0141049c02bfc97ef236ce6d8fe5d94013c721e915982acd2b12b65d9b7d59e20a842005f8fc4e02532e873d37b96f09d6d4511ada8f14042f46614a4c70c0f14beff5ffffffff02404b4c00000000001976a9141aa0cd1cbea6e7458a7abad512a9d9ea1afb225e88ac80fae9c7000000001976a9140eab5bea436a0484cfab12485efda0b78b4ecc5288ac00000000
```


In [27]:
# Exercise 10.1

from io import BytesIO

from network import NetworkEnvelope

stream = BytesIO(bytes.fromhex('f9beb4d974780000000000000000000002010000e293cdbe01000000016dbddb085b1d8af75184f0bc01fad58d1266e9b63b50881990e4b40d6aee3629000000008b483045022100f3581e1972ae8ac7c7367a7a253bc1135223adb9a468bb3a59233f45bc578380022059af01ca17d00e41837a1d58e97aa31bae584edec28d35bd96923690913bae9a0141049c02bfc97ef236ce6d8fe5d94013c721e915982acd2b12b65d9b7d59e20a842005f8fc4e02532e873d37b96f09d6d4511ada8f14042f46614a4c70c0f14beff5ffffffff02404b4c00000000001976a9141aa0cd1cbea6e7458a7abad512a9d9ea1afb225e88ac80fae9c7000000001976a9140eab5bea436a0484cfab12485efda0b78b4ecc5288ac00000000'))
# parse the stream in a NetworkEnvelope
# print the object
print(NetworkEnvelope.parse(stream))

tx          : 01000000016dbddb085b1d8af75184f0bc01fad58d1266e9b63b50881990e4b40d6aee3629000000008b483045022100f3581e1972ae8ac7c7367a7a253bc1135223adb9a468bb3a59233f45bc578380022059af01ca17d00e41837a1d58e97aa31bae584edec28d35bd96923690913bae9a0141049c02bfc97ef236ce6d8fe5d94013c721e915982acd2b12b65d9b7d59e20a842005f8fc4e02532e873d37b96f09d6d4511ada8f14042f46614a4c70c0f14beff5ffffffff02404b4c00000000001976a9141aa0cd1cbea6e7458a7abad512a9d9ea1afb225e88ac80fae9c7000000001976a9140eab5bea436a0484cfab12485efda0b78b4ecc5288ac00000000


### Exercise 11

Use https://bitnodes.earn.com/nodes/ to find nodes to connect to.

#### 11.1. Run the code below and parse some messages

#### Bonus: Run the connect.py script in the terminal to see how asynchronous connections can work

In [28]:
import socket
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
msg = bytes.fromhex('f9beb4d976657273696f6e0000000000650000005f1a69d2721101000100000000000000bc8f5e5400000000010000000000000000000000000000000000ffffc61b6409208d010000000000000000000000000000000000ffffcb0071c0208d128035cbc97953f80f2f5361746f7368693a302e392e332fcf05050001')
s.connect(('46.101.99.121', 8333))
s.sendall(msg)
data = s.recv(200)
print(data.hex())

f9beb4d976657273696f6e00000000006600000038e841567f1101000d040000000000002183215b00000000010000000000000000000000000000000000ffffc61b6409208d0d04000000000000000000000000000000000000000000000000461af2d217758f72102f5361746f7368693a302e31362e302fd40b080001f9beb4d976657261636b000000000000000000005df6e0e2
